In [ ]:
# fetch_beijing_temp_halfhour.py
import sys, json, math, datetime as dt
import numpy as np
import urllib.request
import urllib.parse
import os

# ---- 1) 设置时间范围：最近100天（不含今天），北京时区 ----
# Open-Meteo 的历史接口通常有2~5天延迟，若失败可把 end_date 往前再推几天
TZ = "Asia/Shanghai"
# today = dt.date.today()
# end_date = today - dt.timedelta(days=1)          # 昨天
end_date = dt.date(2025, 9, 20)
day_last = 799
start_date = end_date - dt.timedelta(days=day_last)    # 含起止共100天

data_Folder = "Simulated_data\\"
os.makedirs(data_Folder,exist_ok=True)


In [2]:

# ---- 2) 构造 Open-Meteo 历史API请求（ERA5 或 IFS，免 key）----
lat, lon = 39.9042, 116.4074  # 北京
base = "https://archive-api.open-meteo.com/v1/era5"
params = {
    "latitude": f"{lat:.4f}",
    "longitude": f"{lon:.4f}",
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "hourly": "temperature_2m",
    "timezone": TZ,
    "temperature_unit": "celsius",
}
url = base + "?" + urllib.parse.urlencode(params)
print("Requesting:", url)

# ---- 3) 拉取每小时温度 JSON ----
with urllib.request.urlopen(url, timeout=60) as r:
    data = json.load(r)

# 基本校验
hourly = data.get("hourly", {})
times = hourly.get("time", [])
temps = hourly.get("temperature_2m", [])
if not times or not temps or len(times) != len(temps):
    raise RuntimeError("API 返回异常或无数据")

# 按时间排序（通常已排序，这里稳妥起见）
pairs = sorted(zip(times, temps))
times_sorted = [t for t, _ in pairs]
temps_sorted = np.array([v for _, v in pairs], dtype=float)

# 计算应有的小时点个数：100天 * 24 = 2400
expected_hours = day_last * 24
if len(temps_sorted) < expected_hours:
    print(f"Warning: 返回小时数 {len(temps_sorted)} < 预期 {expected_hours}，"
          "可能因近几天数据延迟。可把 end_date 再往前推几天重试。", file=sys.stderr)
print(len(temps_sorted))


Requesting: https://archive-api.open-meteo.com/v1/era5?latitude=39.9042&longitude=116.4074&start_date=2024-11-25&end_date=2025-09-20&hourly=temperature_2m&timezone=Asia%2FShanghai&temperature_unit=celsius
7200


In [ ]:
# ---- 4) 用线性插值把“每小时” → “每半小时”（4800点）----
# 小时索引：0,1,2,...,H-1
H = len(temps_sorted)
x = np.arange(H, dtype=float)

# 目标半小时索引：0,0.5,1.0,1.5,...,H-1.0
x_half = np.arange(0, H, 0.5, dtype=float)
# 线性插值
y_half = np.interp(x_half, x, temps_sorted)

# 若恰好等于100天，则应为 4800 点
print("Half-hourly points:", y_half.size)

# ---- 5) 保存为 npy 文件 ----
np.save(data_Folder+"beijing_temp_halfhour_"+str(day_last)+".npy", y_half)
print("data\\Saved: beijing_temp_halfhour.npy")


Half-hourly points: 14400
data\Saved: beijing_temp_halfhour.npy
